In [124]:
using Plots, LinearAlgebra

Plots.PyPlotBackend()

# CIRCUS
This notebook is just for trying out some ideas for the final algorithm.
Let's use as example the following case:
$$
\begin{aligned}
\textrm{Maximize} \quad & x_1 + x_2 \\
\textrm{Subject to} \quad &
        -x_1 + x_2  &\leq 1\\
\quad & x_1        &\leq 3\\
\quad & x_2         &\leq 2\\
\quad & x_1,x_2     &\geq 0
\end{aligned}
$$

We then put this in the standard form by adding slack variables:
$$
\begin{aligned}
\textrm{Maximize} \quad & x_1 + x_2 \\
\textrm{Subject to} \quad &
        -x_1 + x_2 + x_3  &= 1\\
\quad & x_1 + x_4        &= 3\\
\quad & x_2 + x_5         &= 2\\
\quad & x_1,x_2,x_3,x_4,x_5     &\geq 0
\end{aligned}
$$

Therefore we have:
$$
\begin{aligned}
\textrm{Maximize} \quad & c^T {x} \\
\textrm{Subject to} \quad & Ax = b \\
\quad & x\geq 0
\end{aligned}
$$

First, we will implement the Simplex Method following the book "Understanding and Using Linear Programming".

In [327]:
A = [-1 1 1 0 0;
      1 0 0 1 0;
      0 1 0 0 1 ]
c = [1,1,0,0,0]
b = [1,3,2]
# starting basic feasible point
xo = [0,0,1,3,2];

function Simplex(c,A,b,initial_bfs;max_iterations=100)
    # Initial setup
    e  = 10^-5
    B  = findall(xo .> 0+e)
    N  = findall(xo .<= 0+e)
    xn = initial_bfs[N]; xb = initial_bfs[B];
    
    # Simplex pivoting iteration
    for i = 1:max_iterations
        Ab = A[:,B]; An = A[:,N]; cb = c[B]; cn = c[N]
        p  = inv(Ab)*b
        Q  = -inv(Ab)*An
        r  = (cb'*Q + cn')'
        if all(r.<= 0)
            x_final = vcat(hcat(B,p),hcat(N,zeros(length(N))))
            x_final = x_final[sortperm(x_final[:,1]),:]
            return x_final
        end
        zo = cb'*p
#         z  = zo + r'*xn
        index_in =findmax(r)[2]
        x_in = N[index_in]

        if any(Q[:,index_in] .< 0)
        #     index_out = findmin(p./Q[:,index_in])[2]
            coef_entering = -p./Q[:,index_in] 
            q_neg_index   = findall(coef_entering .> 0)
            index_out     =findfirst(coef_entering .== findmin(coef_entering[q_neg_index])[1])
            x_out     = B[index_out]
            B[index_out] = x_in
            N[index_in]  = x_out
        else
            error("Unbounded")
        end
    end
end


Simplex (generic function with 2 methods)

In [282]:
Simplex

([3, 1, 5], [4, 2])

In [283]:
Ab = A[:,B]; An = A[:,N]; cb = c[B]; cn = c[N]
p  = inv(Ab)*b
Q  = -inv(Ab)*An
r  = (cb'*Q + cn')'
zo = cb'*p
z  = zo + r'*xn
index_in =findmax(r)[2]
x_in = N[index_in]

if any(Q[:,index_in] .< 0)
    coef_entering = -p./Q[:,index_in] 
    q_neg_index   = findall(coef_entering .> 0)
    index_out     = findfirst(coef_entering .== findmin(coef_entering[q_neg_index])[1])
    x_out     = B[index_out]
    B[index_out] = x_in
    N[index_in]  = x_out
else
    println("Unbounded")
end

5

In [284]:
B,N

([3, 1, 2], [4, 5])

In [285]:
Ab = A[:,B]; An = A[:,N]; cb = c[B]; cn = c[N]
p  = inv(Ab)*b
Q  = -inv(Ab)*An
r  = (cb'*Q + cn')'
zo = cb'*p
z  = zo + r'*xn


5.0

In [300]:
B

3-element Array{Int64,1}:
 3
 1
 2

In [298]:
hcat(B,p)

3×2 Array{Float64,2}:
 3.0  2.0
 1.0  3.0
 2.0  2.0

In [325]:
# vcat(hcat(B,p)[sortperm(B),:],hcat(N,zeros(length(N))))


x_final = vcat(hcat(B,p),hcat(N,zeros(length(N))))
x_final = x_final[sortperm(x_final[:,1]),:]

5×2 Array{Float64,2}:
 1.0  3.0
 2.0  2.0
 3.0  2.0
 4.0  0.0
 5.0  0.0

In [278]:
x_out

5

In [280]:
r

2-element Array{Float64,1}:
 -1.0
  1.0